In [17]:
import datetime
import os

In [1]:
cancerList = ["anus","bone","brain","breast","cervix","colorectal","esophagus","eyeAndOrbit","gallbladderAndBiliaryTract","headAndNeck",
            "kaposiSarcoma","kidney","liver","lung","lymphoidNeoplasms","melanoma","mesothelioma","myeloidNeoplasms","otherDigestive",
           "otherEndocrine","otherFemaleGenital","otherMaleGenital","otherNervousSystem","otherRespiratory","otherUrinaryOrgans",
           "ovary","pancreas","prostate","smallIntestine","softTissueSarcoma","stomach","tCellAndNKCellNeoplasms","testis","thyroid",
           "urinaryBladder","uterus"]

In [47]:
### 获取不同癌症MPC样本
sampleCountDict = {}
for cancer in cancerList:
    with open("/home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/04.cancerGroup/03.whiteData/"+cancer+".filtered.white.sample","r") as fi:
        for line in fi:
            sampleID = line.strip()
            sampleCountDict[sampleID] = sampleCountDict.get(sampleID,0)+1
    fi.close()

fo = open("/home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/01.multiDiffPcancer/diffCancer.filtered.white.sample","w")
for key,value in sampleCountDict.items():
    if value >= 2:
        fo.write(key+"\n")
fo.close()

In [50]:
### 提取各种多原发癌患病次数
# 2\3\4\5\6种癌症样本
fo2 = open("/home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/03.multiPcancer/MPC.2cancers.sample","w")
fo3 = open("/home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/03.multiPcancer/MPC.3cancers.sample","w")
fo4 = open("/home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/03.multiPcancer/MPC.4cancers.sample","w")
fo5 = open("/home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/03.multiPcancer/MPC.5cancers.sample","w")
fo6 = open("/home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/03.multiPcancer/MPC.6cancers.sample","w")
for key,value in sampleCountDict.items():
    if value == 2:
        fo2.write(key+"\n")
    elif value == 3:
        fo3.write(key+"\n")
    elif value == 4:
        fo4.write(key+"\n")
    elif value == 5:
        fo5.write(key+"\n")
    elif value == 6:
        fo6.write(key+"\n")
fo2.close()
fo3.close()
fo4.close()
fo5.close()
fo6.close()

In [10]:
### 获取相同癌症多原发癌，病理组织是否相同
def dateSubtraction(date1,date2):
    d0 = datetime.datetime.strptime(date1, "%Y-%m-%d")
    df = datetime.datetime.strptime(date2, "%Y-%m-%d")
    return (df-d0).days

# 获取相同癌症多原发癌样本
ICD10CancerList = []
for i in range(98):
    for j in ["","0","1","2","3","4","5","6","7","8","9"]:
        ICD10CancerList.append("C{:0>2d}".format(i)+j)
for i in range(10):
    for j in ["","0","1","2","3","4","5","6","7","8","9"]:
        ICD10CancerList.append("D{:0>2d}".format(i)+j)

sampleDict = {}
sameDiseaseCount = []
with open('/home/luohh/UKB50wData/01.Download/04.PhenotypeData/ukb670790.txt',"r",encoding='gbk') as fi:
    header = fi.readline().strip("\n").lstrip().split("\t")
    X40006List = []
    for index in range(len(header)):
        if "40006" == header[index].split("-")[0]:
            X40006List.append(index)
    for line in fi:
        data = line.strip("\n").lstrip().split("\t")
        try:
            diseaseCount = {}
            n = 0
            targetData1 = [data[index] for index in X40006List]
            for i in targetData1:
                diseaseCount[i] = diseaseCount.get(i,0) + 1
            for key,value in diseaseCount.items():
                if key == "":
                    continue
                else:
                    if value > 1 and key in ICD10CancerList:
                        n += 1
                        sampleDict[data[0]] = key
            sameDiseaseCount.append(n)
        except:
            print(data)
fi.close()

# 获取相同癌症多原发癌信息，用于判断
fo = open("/home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/02.multiSamePcancer/sameCancer.sample","w")
fo.write("\t".join(("sampleID","cancerCode","histologyCT","gapDays"))+"\n")

with open('/home/luohh/UKB50wData/01.Download/04.PhenotypeData/ukb670790.txt',"r",encoding='gbk') as fi:
    header = fi.readline().strip("\n").lstrip().split("\t")
    X40005List = []
    for index in range(len(header)):
        if "40005" == header[index].split("-")[0]:
            X40005List.append(index)
            
    X40006List = []
    for index in range(len(header)):
        if "40006" == header[index].split("-")[0]:
            X40006List.append(index)
            
    X40011List = []
    for index in range(len(header)):
        if "40011" == header[index].split("-")[0]:
            X40011List.append(index)
            
    for line in fi:
        data = line.strip("\n").lstrip().split("\t")
        if data[0] not in sampleDict:
            continue
#         try:
        targetData1 = [data[index] for index in X40005List]
        targetData2 = [data[index] for index in X40006List]
        targetData3 = [data[index] for index in X40011List]
        cancerIndexList = []
        for i in range(len(targetData2)):
            if targetData2[i] == sampleDict[data[0]]:
                cancerIndexList.append(i)
        diagnoseDate = [targetData1[index] for index in cancerIndexList]
        diagnoseDate.sort()
        gapDays = dateSubtraction(diagnoseDate[0],diagnoseDate[1])

        histologyCT = [targetData3[index] for index in cancerIndexList]
        if len(set(histologyCT)) == 1:
            fo.write("\t".join((data[0],sampleDict[data[0]],"same",str(gapDays)))+"\n")
        else:
            fo.write("\t".join((data[0],sampleDict[data[0]],"diff",str(gapDays)))+"\n")
#         except:
#             print(data)
fi.close()

KeyboardInterrupt: 

In [51]:
### 过滤低质量样本
remainSampleDict = {}
with open("/home/luohh/UKB50wData/04.PhenoDataExtract/01.SampleProcess/remain.sample","r") as fi:
    for line in fi:
        sampleid = line.strip()
        remainSampleDict[sampleid] = 0
fi.close()

fo = open("/home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/02.multiSamePcancer/sameCancer.filtered.sample","w")
with open("/home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/02.multiSamePcancer/sameCancer.sample","r") as fi:
    fi.readline()
    for line in fi:
        data = line.strip().split()
        if data[0] in remainSampleDict:
            fo.write(line)
fi.close()
fo.close()

In [68]:
### 获取相同癌症MPC样本
removalList = []
for i in ["","0","1","2","3","4","5","6","7","8","9"]:
    removalList.append("C44"+i)
    removalList.append("C77"+i)
    removalList.append("C78"+i)
    removalList.append("C79"+i)
    removalList.append("C80"+i)
    removalList.append("D04"+i)
removalList.append("C857")
removalList.append("C859")
removalList.append("C880")
removalList.append("C969")

### 提取白人癌症样本
whitePeopleDict = {}
with open("/home/luohh/UKB50wMultiPcancer/01.data/02.phenoData/basic.pheno","r") as fi:
    for line in fi:
        data = line.strip().split()
        if data[4] in ["1","1001","1002","1003"]:
            whitePeopleDict[data[0]] = 1
fi.close()

### 过滤其它癌症编码
fo = open("/home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/02.multiSamePcancer/sameCancer.filtered.filterOtherCode.sample","w")
with open("/home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/02.multiSamePcancer/sameCancer.filtered.sample","r") as fi:
    for line in fi:
        data = line.strip().split()
        if data[1] not in removalList:
            fo.write(line)
fi.close()
fo.close()

### 保留白人样本
fo = open("/home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/02.multiSamePcancer/sameCancer.filtered.filterOtherCode.white.sample","w")
with open("/home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/02.multiSamePcancer/sameCancer.filtered.filterOtherCode.sample","r") as fi:
    for line in fi:
        data = line.strip().split()
        if data[0] in whitePeopleDict:
            fo.write(line)
fi.close()
fo.close()

In [69]:
### 过滤掉不同多原发癌里已经鉴定到的样本
diffPCancerDict = {}
with open("/home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/01.multiDiffPcancer/diffCancer.filtered.white.sample","r") as fi:
    for line in fi:
        diffPCancerDict[line.strip()] = 1
fi.close()

fo = open("/home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/02.multiSamePcancer/sameCancer.filtered.filterOtherCode.white.removeDiffMPC.sample","w")
with open("/home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/02.multiSamePcancer/sameCancer.filtered.filterOtherCode.white.sample","r") as fi:
    for line in fi:
        data = line.strip().split()
        if data[0] not in diffPCancerDict:
            fo.write(line)
fo.close()
fi.close()

In [70]:
### 提取患有相同癌症MPC，病理组织相同，发病间隔大于3年的癌症样本
fo = open("/home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/02.multiSamePcancer/sameCancer.same.gt3years.sample","w")
with open("/home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/02.multiSamePcancer/sameCancer.filtered.filterOtherCode.white.removeDiffMPC.sample","r") as fi:
    fi.readline()
    for line in fi:
        data = line.strip().split()
        if data[2] == "same" and int(data[3]) > 1095:
            fo.write(line)
fi.close()
fo.close()

In [71]:
### 提取患有相同癌症MPC，病理组织不同癌症样本
fo = open("/home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/02.multiSamePcancer/sameCancer.diff.sample","w")
with open("/home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/02.multiSamePcancer/sameCancer.filtered.filterOtherCode.white.removeDiffMPC.sample","r") as fi:
    fi.readline()
    for line in fi:
        data = line.strip().split()
        if data[2] == "diff":
            fo.write(line)
fi.close()
fo.close()

In [72]:
os.system("cat /home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/02.multiSamePcancer/sameCancer.diff.sample /home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/02.multiSamePcancer/sameCancer.same.gt3years.sample > /home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/02.multiSamePcancer/sameCancer.all.sample")
os.system("awk '{print $1}' /home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/02.multiSamePcancer/sameCancer.all.sample > /home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/02.multiSamePcancer/sameCancer.all.sampleid")

0

In [73]:
os.system("cat /home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/01.multiDiffPcancer/diffCancer.filtered.white.sample /home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/02.multiSamePcancer/sameCancer.all.sampleid > /home/luohh/UKB50wMultiPcancer/01.data/04.sampleData/03.multiPcancer/multiPCancer.sample")

0

In [46]:
### 提取MHC区域的rsid
fo = open("/home/luohh/UKB50wMultiPcancer/01.data/05.genotypeQC/MHC.exclude.var","w")
with open("/home/luohh/UKB50wData/01.Download/05.InfoData/03.ImputationMAFandInfo/ukb_mfi_chr6_v3.txt","r") as fi:
    for line in fi:
        data = line.strip().split()
        if int(data[2]) > 28477797 and int(data[2]) < 33448354:
            fo.write(data[1]+"\n")
fi.close()
fo.close()